In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [28]:
batch_size = 128
num_epochs = 200
extract_size = 2048
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
encoder = Encoder()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train.h')

### init model

In [5]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [6]:
classify_model.load_state_dict(torch.load('./models/class_multi_model_last.pkl'))

### time to train model

In [26]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(1, num_epochs+1):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multi_model_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multi_model_last.pkl'))

Epoch [1/200], Step [240/255], Loss: 0.5811, Accuracy: 75.00%, Best_acc: 89.06%            
 Epoch 1, spent time:5.99s
Epoch [2/200], Step [240/255], Loss: 0.4283, Accuracy: 83.59%, Best_acc: 89.84%            
 Epoch 2, spent time:5.96s
Epoch [3/200], Step [240/255], Loss: 0.5388, Accuracy: 83.59%, Best_acc: 89.84%            
 Epoch 3, spent time:6.01s
Epoch [4/200], Step [240/255], Loss: 0.5300, Accuracy: 78.91%, Best_acc: 89.84%            
 Epoch 4, spent time:6.05s
Epoch [5/200], Step [240/255], Loss: 0.5100, Accuracy: 77.34%, Best_acc: 89.84%            
 Epoch 5, spent time:5.99s
Epoch [6/200], Step [240/255], Loss: 0.5031, Accuracy: 82.03%, Best_acc: 89.84%            
 Epoch 6, spent time:5.98s
Epoch [7/200], Step [240/255], Loss: 0.4705, Accuracy: 83.59%, Best_acc: 89.84%            
 Epoch 7, spent time:5.98s
Epoch [8/200], Step [240/255], Loss: 0.5284, Accuracy: 82.81%, Best_acc: 89.84%            
 Epoch 8, spent time:6.00s
Epoch [9/200], Step [240/255], Loss: 0.5614, Acc

Epoch [69/200], Step [240/255], Loss: 0.4407, Accuracy: 82.81%, Best_acc: 92.19%            
 Epoch 69, spent time:6.01s
Epoch [70/200], Step [240/255], Loss: 0.4441, Accuracy: 86.72%, Best_acc: 92.19%            
 Epoch 70, spent time:6.02s
Epoch [71/200], Step [240/255], Loss: 0.4029, Accuracy: 84.38%, Best_acc: 92.19%            
 Epoch 71, spent time:5.97s
Epoch [72/200], Step [240/255], Loss: 0.4406, Accuracy: 81.25%, Best_acc: 92.19%            
 Epoch 72, spent time:6.05s
Epoch [73/200], Step [240/255], Loss: 0.5997, Accuracy: 81.25%, Best_acc: 92.19%            
 Epoch 73, spent time:5.94s
Epoch [74/200], Step [240/255], Loss: 0.4863, Accuracy: 82.03%, Best_acc: 92.19%            
 Epoch 74, spent time:5.98s
Epoch [75/200], Step [240/255], Loss: 0.4404, Accuracy: 83.59%, Best_acc: 92.19%            
 Epoch 75, spent time:5.99s
Epoch [76/200], Step [240/255], Loss: 0.4226, Accuracy: 83.59%, Best_acc: 92.19%            
 Epoch 76, spent time:5.99s
Epoch [77/200], Step [240/255], 

Epoch [136/200], Step [240/255], Loss: 0.4493, Accuracy: 78.12%, Best_acc: 94.53%            
 Epoch 136, spent time:5.98s
Epoch [137/200], Step [240/255], Loss: 0.4685, Accuracy: 85.16%, Best_acc: 94.53%            
 Epoch 137, spent time:5.92s
Epoch [138/200], Step [240/255], Loss: 0.5121, Accuracy: 77.34%, Best_acc: 94.53%            
 Epoch 138, spent time:5.86s
Epoch [139/200], Step [240/255], Loss: 0.5007, Accuracy: 79.69%, Best_acc: 94.53%            
 Epoch 139, spent time:5.95s
Epoch [140/200], Step [240/255], Loss: 0.3950, Accuracy: 88.28%, Best_acc: 94.53%            
 Epoch 140, spent time:5.92s
Epoch [141/200], Step [240/255], Loss: 0.5471, Accuracy: 82.03%, Best_acc: 94.53%            
 Epoch 141, spent time:5.97s
Epoch [142/200], Step [240/255], Loss: 0.4267, Accuracy: 81.25%, Best_acc: 94.53%            
 Epoch 142, spent time:6.00s
Epoch [143/200], Step [240/255], Loss: 0.4919, Accuracy: 81.25%, Best_acc: 94.53%            
 Epoch 143, spent time:5.96s
Epoch [144/200],

### How good is the model?

In [31]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_new.h")
valid_data_loader = get_encoder_loader(transform_train, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_new.h")

encoding 4981/4982->99.98%, spent_time:5:28.78

In [33]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 64.91%.
